### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size
import time


handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)


### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40).eval()
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)
new_state_dict = OrderedDict()
for key in torch_model.O.layers[1].state_dict():
#     print(torch_model.O.layers[1].state_dict()[key].shape)
    if key != "num_batches_tracked":
        new_state_dict[key] = torch.rand(40)

torch_model.O.layers[1].load_state_dict(new_state_dict)
print(torch_model.O.layers[1].state_dict())
# for name, submodule in torch_model.named_modules():
#     if name != "":
#         print(f"{name}: {submodule}")

OrderedDict([('weight', tensor([0.14819485, 0.10776299, 0.18915886, 0.07321286, 0.08802354, 0.95936340,
        0.32395267, 0.09556627, 0.99297142, 0.97439438, 0.52161252, 0.19358766,
        0.66018587, 0.44498897, 0.60995883, 0.10150993, 0.45451957, 0.52531058,
        0.93877780, 0.28106594, 0.14415997, 0.29765218, 0.96479279, 0.84275991,
        0.47767246, 0.47202319, 0.24503112, 0.77267098, 0.72283816, 0.95040357,
        0.48314852, 0.29976338, 0.51107556, 0.39307076, 0.49783385, 0.69130814,
        0.25721705, 0.40953445, 0.73893833, 0.36656791])), ('bias', tensor([0.26053399, 0.07725668, 0.47410673, 0.22577035, 0.87139207, 0.39811850,
        0.86413956, 0.69497591, 0.05985016, 0.87185097, 0.53687239, 0.02068603,
        0.82220358, 0.16684115, 0.60327083, 0.40372521, 0.86406064, 0.62994492,
        0.24517769, 0.38250852, 0.09975529, 0.67661357, 0.23945385, 0.28818518,
        0.00484180, 0.87734479, 0.35381472, 0.77792144, 0.49775392, 0.64469302,
        0.29402006, 0.239041

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
# forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
# forward_dict["res_block"] = "ResidualBlock"
# forward_dict["R2"] = "EdgeBlock"

In [4]:
common_dim = 5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_attr": 3,
        "node_dim": common_dim,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [5]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<52,20>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid', #sigmoid
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<52,20>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
pyg_to_hls config: {'OutputDir': 'test_GNN', 'ProjectName': 'myproject', 'Backend': 'Vivado', 'XilinxPart': 'xcku115-flvb2104-2-i', 'Board': None, 'ClockPeriod': 5, 'IOType': 'io_parallel', 'HLSConfig': {'Model': {'Precision': 'ap_fixed<52,20>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}, 'PytorchModel': InteractionNetwork(), 'InputShape': {'NodeAttr': [28, 3], 'EdgeAttr': [37, 4], 'EdgeIndex': [37, 2], 'NodeDim': 5, 'EdgeDim': 5}, 'ForwardDictionary': OrderedDict([('node_encoder', 'NodeEncoder'), ('edge_encoder', 'EdgeEncoder'), ('O', 'NodeBlock')]), 'ActivateFinal': 'sigmoid', 'ParallelizationFactor': 16, 'gnn_resource_limit': 'false'}
PygModelReader node_dim: 5
PygModelReader edge_dim: 5
PygModelReader node_attr: 3
PygModelReader edge_attr: 4
node encoder layer_dict['n_in']: 3
node encoder layer_dict['n_out']: 5
node_encoder la

## hls_model.compile() builds the C-function for the model

In [7]:
hls_model.compile()

Writing HLS project
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
def_cpp: layer5_t layer5_out[N_LAYER_1_5*N_LAYER_2_5]
def_cpp: layer6_t layer6_out[N_NODE*LAYER6_OUT_DIM]
def_cpp: layer7_t layer7_out[N_LAYER_1_4*LAYER7_OUT_DIM]
Done
lib_name: firmware/myproject-A43ecFd9.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [8]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [9]:
data = graphs[0]
# print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
# print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
# print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
time.sleep(14)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")
# print((np.abs(torch_pred- hls_pred)))
# print(torch_pred)
# print(hls_pred)

<class 'list'>
hls_pred.shape: (140,)
MSE: 4.562617505143862e-06


In [10]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [11]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
    print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"MSE_l: {MSE_l}")
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

MSE: 4.562617505143862e-06
MSE: 4.4236076064407825e-06
MSE: 1.6629373931209557e-05
MSE: 4.64712647954002e-05
MSE: 4.704269940702943e-06
MSE: 4.216642992105335e-06
MSE: 4.249519861332374e-06
MSE: 2.8564481908688322e-05
MSE: 4.582159363053506e-06
MSE: 4.726725364889717e-06
MSE_l: [4.5626175e-06 4.4236076e-06 1.6629374e-05 4.6471265e-05 4.7042699e-06
 4.2166430e-06 4.2495199e-06 2.8564482e-05 4.5821594e-06 4.7267254e-06]
overall MSE: 1.231306669069454e-05
(10,)


In [12]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

(28, 3)
(37, 4)
(37, 2)
torch.Size([2, 37])


In [13]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =100000000): #n_graphs =1000
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array

        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        else:
            print("bad graph")
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [14]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7fda8ceba290>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7fda8cd5f490>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7fda8cd5f510>}


In [15]:
# from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
# node_scaler = StandardScaler()
# edge_scaler = StandardScaler()
# for stage in [stages[0]]:
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
#     print(type(graphs))

#     for data in graphs: # fit data to scaler b4 feedforwarding
# #         print(f"data type: {type(data)}")
#         node_scaler.fit(data.x) 
#         edge_scaler.fit(data.edge_attr) 
    
    for data in graphs:
       
#         data.x = torch.from_numpy(node_scaler.transform(data.x)).float()
#         data.edge_attr = torch.from_numpy(edge_scaler.transform(data.edge_attr)).float()
#         data.hls_data[0] = node_scaler.transform(data.hls_data[0])
#         data.hls_data[1] = edge_scaler.transform(data.hls_data[1])
        
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
#         print(f"torch_pred shape: {torch_pred.shape}")
        hls_pred = hls_model.predict(data.hls_data)
#         print(f"hls_pred shape: {hls_pred.shape}")
        MSE = mean_squared_error(torch_pred, hls_pred)
#         print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
#         break
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")

n_graphs: 332406
writing test bench data for 1st graph
n_graphs: 71226
writing test bench data for 1st graph
n_graphs: 175113
writing test bench data for 1st graph
MSE_l :[6.0180295e-03 3.0593721e-03 3.5855030e-06 ... 6.7110254e-06 2.5671576e-05
 4.3877955e-03]
MSE means: 0.0053800009191036224


In [16]:
MSE_l.shape

(578745,)

In [17]:
# torch_model.state_dict()
with open('MSE_l.npy', 'wb') as f:
    np.save(f, MSE_l)